# Compute geology indices

In [ ]:
%matplotlib inline  
import os, sys
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import scripts.utility as util

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])

## Reading LiMW2015 and GLHYMPS geopackage data

In [ ]:
catchment_name = 'camels' #  CONUS_HUC12 or camels
saveCSV = True
saveNetCDF = True

In [ ]:
src_dir = '/glade/u/home/mizukami/proj/cmip6_hydro/camels/attributes/ingredient/geology'
LiMW2015_gpkg_list = [os.path.join(src_dir, 'LiMW2015_North_America_WGS84.gpkg')]
GLHYMPS_gpkg_list = [os.path.join(src_dir, 'GLHYMPS_North_America_WGS84.gpkg')]

# camels basin
gpkg = '/glade/campaign/ral/hap/mizukami/proj/cmip6_hydro/geospatial/gagesII_671_shp_geogr.gpkg' #HCDN_nhru_final_671.buff_fix_holes.CAMELSandTDX_areabias_fix.simp0.001.level1.gpkg
LiMW2015_mapping_file = '/glade/u/home/mizukami/proj/cmip6_hydro/camels/attributes/scripts/gen_mapping/weight_file/spatialweights_LiMW2015_to_camels.nc'
GLHYMPS_mapping_file = '/glade/u/home/mizukami/proj/cmip6_hydro/camels/attributes/scripts/gen_mapping/weight_file/spatialweights_GLHYMPS_to_camels.nc'
# HUC12
#gpkg = '/glade/campaign/ral/hap/mizukami/proj/cmip6_hydro/geospatial/conus_HUC12_merit_v7b_simplified0.001.gpkg'
#LiMW2015_mapping_file = '/glade/u/home/mizukami/proj/cmip6_hydro/camels/attributes/scripts/gen_mapping/weight_file/spatialweights_LiMW2015_to_CONUS_HUC12.nc'
#GLHYMPS_mapping_file = '/glade/u/home/mizukami/proj/cmip6_hydro/camels/attributes/scripts/gen_mapping/weight_file/spatialweights_GLHYMPS_to_CONUS_HUC12.nc'


In [ ]:
# netcdf header: variable name
LiMW2015_attr_dict = {
    'OBJECTID':'numeric polygon ID',
    'IDENTITY_':'ID',
    'xx':'The first level lithological class ',
}

GLHYMPS_attr_dict = {
    'OBJECTID':'numeric polygon ID',
    'IDENTITY_':'ID',
    'Porosity':'subsurface porosity',
    'Permeability_permafrost':'near-surface permeability (log10) with permafrost effects',
    'Permeability_no_permafrost':'near-surface permeability (log10) neglecting permafrost effects'
} # use Permeability_permafrost

In [ ]:
# target catchment attribute
catch_attrs = {
    'CONUS_HUC12': ['HUCIDXint'],
    'camels': ['GAGE_ID']
}

In [ ]:
GLiM_class = {
    "su":{"class":"Unconsolidated sediments", "color":"lightgoldenrod1"},
    "ss":{"class":"Siliciclastic sedimentary rocks", "color":"goldenrod"},
    "sm":{"class":"Mixed sedimentary rocks", "color":"burlywood4"},
    "py":{"class":"Pyroclastics", "color":"tan4"},
    "sc":{"class":"Carbonate sedimentary rocks", "color":"steelblue3"},
    "ev":{"class":"Evaporites", "color":"olivedrab1"},
    "mt":{"class":"Metamorphics", "color":"palegreen3"},
    "pa":{"class":"Acid plutonic rocks", "color":"brown3"},
    "pi":{"class":"Intermediate plutonic rocks", "color":"violetred4"},
    "pb":{"class":"Basic plutonic rocks", "color":"purple4"},
    "va":{"class":"Acid volcanic rocks", "color":"pink"},
    "vi":{"class":"Intermediate volcanic rocks", "color":"orchid2"},
    "vb":{"class":"Basic volcanic rocks", "color":"magenta3"},
    "ig":{"class":"Ice and glaciers", "color":"paleturquoise1"},
    "wb":{"class":"Water bodies", "color":"turquoise2"},
    "nd":{"class":"No Data", "color":"slategray"},
}

In [ ]:
%%time
print(f'Reading LiMW2015 geology data')
gdf_LiMW2015 = util.read_shps(LiMW2015_gpkg_list,list(LiMW2015_attr_dict.keys()))
ds_LiMW2015 = gdf_LiMW2015.rename(columns={'OBJECTID':'hru'}).set_index('hru').to_xarray()
ds_LiMW2015 = ds_LiMW2015.rename({'xx':'lithological_class'})
ds_LiMW2015['carbonate_rock'] = xr.DataArray(np.ones(len(ds_LiMW2015['hru']), dtype=int), dims=ds_LiMW2015.dims, coords=ds_LiMW2015.coords)
ds_LiMW2015['carbonate_rock'] = ds_LiMW2015['carbonate_rock'].where(ds_LiMW2015['lithological_class']=='sc', 0)
ds_LiMW2015

In [ ]:
%%time
print(f'Reading GLHYMPS geology data')
gdf_GLHYMPS = util.read_shps(GLHYMPS_gpkg_list,list(GLHYMPS_attr_dict.keys()))
ds_GLHYMPS = gdf_GLHYMPS.rename(columns={'OBJECTID':'hru'}).set_index('hru').to_xarray()
ds_GLHYMPS

## Re-mapping
- 1st dominant class
- 2nd cominant class
- porosity
- Permeability

In [ ]:
%%time
dr_mask = xr.where(np.isnan(ds_LiMW2015['carbonate_rock']),0,1)
a = util.remap_mode(xr.open_dataset(LiMW2015_mapping_file), ds_LiMW2015, dr_mask, ['lithological_class'])
a['1st_dominant_lithological_class_fraction'] = a['1st_dominant_lithological_class_fraction'].where(a['1st_dominant_lithological_class']!='N/A', 0)
a['2nd_dominant_lithological_class_fraction'] = a['2nd_dominant_lithological_class_fraction'].where(a['2nd_dominant_lithological_class']!='N/A', 0)

In [ ]:
%%time
# carbonate rock fraction
a1 = util.remap_mean(xr.open_dataset(LiMW2015_mapping_file), ds_LiMW2015, dr_mask, ['carbonate_rock'])
a = xr.merge([a, a1['carbonate_rock'].rename('carbonate_rock_frac')])

In [ ]:
%%time
dr_mask = xr.where(np.isnan(ds_GLHYMPS['Porosity']),0,1)
a1 = util.remap_mean(xr.open_dataset(GLHYMPS_mapping_file), ds_GLHYMPS, dr_mask, ['Porosity', 'Permeability_permafrost'])
a = xr.merge([a, a1])
a

## Dataset to Dataframe

In [ ]:
df = a.to_dataframe()

## Save in csv or netcdf

In [ ]:
if saveCSV:
    df.to_csv(f'{catchment_name}_geol.csv', float_format='%g')
if saveNetCDF:
    a.to_netcdf(f'{catchment_name}_geol.nc')

## Plotting

In [ ]:
%%time
gdf_camels = util.read_shps([gpkg],catch_attrs[catchment_name])

In [ ]:
gdf_camels = gdf_camels.merge(a[['2nd_dominant_lithological_class',
                                 '2nd_dominant_lithological_class_fraction',
                                 '1st_dominant_lithological_class', 
                                 '1st_dominant_lithological_class_fraction',
                                 'carbonate_rock_frac',
                                 'Porosity', 
                                 'Permeability_permafrost']].to_dataframe(),left_on=catch_attrs[catchment_name], right_index=True)

In [ ]:
var_name = '1st_dominant_lithological_class'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                legend=True, legend_kwds={'fontsize':'xx-small','loc':'best'},
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_geol_{var_name}.png', dpi=300)

In [ ]:
var_name = '1st_dominant_lithological_class_fraction'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_geol_{var_name}.png', dpi=300)

In [ ]:
var_name = 'carbonate_rock_frac'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_geol_{var_name}.png', dpi=300)

In [ ]:
var_name = 'Porosity'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(0, 0.25),
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_geol_{var_name}.png', dpi=300)

In [ ]:
var_name = 'Permeability_permafrost'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(-20, -10),
                legend=True
               );
ax.set_title(f'{var_name} log90');
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_geol_{var_name}.png', dpi=300)